In [1]:
import torch
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import wandb

C:\Users\kosto\miniconda3\envs\work\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\kosto\miniconda3\envs\work\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
train_dataset = torchvision.datasets.MNIST(".", train=True, download=True)
test_dataset = torchvision.datasets.MNIST(".", train=False, download=True)

In [3]:
def draw_int(image):
    display(Image.fromarray(np.array(image)))

def draw_float(image):
    display(Image.fromarray((np.array(image) * 255).astype(np.uint8)))

In [4]:
print(train_dataset.data.shape, train_dataset.targets.shape)
draw_int(train_dataset.data[0]), train_dataset.targets[0]

torch.Size([60000, 28, 28]) torch.Size([60000])


(None, tensor(5))

In [5]:
NOISE_FACTOR = 0.4

def preprocess(array):
    array = np.array(array)
    array = array.astype(np.float32) / 255.0
    return array#.reshape(len(array), 28, 28)

def generate_noise(arr):
    return np.clip(
        arr + NOISE_FACTOR * np.random.normal(0.0, 1.0, size=arr.shape),
        0.0, 1.0
    )

In [6]:
X_train_clean = preprocess(train_dataset.data)
X_train_noise = generate_noise(X_train_clean)
y_train = train_dataset.targets

In [7]:
X_test_clean = preprocess(test_dataset.data)
X_test_noise = generate_noise(X_test_clean)
y_test = test_dataset.targets

In [8]:
draw_float(X_test_clean[0]), draw_float(X_test_noise[0])

(None, None)

In [9]:
X_train_tensor_clean = torch.tensor(X_train_clean, dtype=torch.float32).cuda().unsqueeze(1)
X_train_tensor_noise = torch.tensor(X_train_noise, dtype=torch.float32).cuda().unsqueeze(1)

X_test_tensor_clean = torch.tensor(X_test_clean, dtype=torch.float32).cuda().unsqueeze(1)
X_test_tensor_noise = torch.tensor(X_test_noise, dtype=torch.float32).cuda().unsqueeze(1)

In [10]:
train_set = torch.utils.data.TensorDataset(X_train_tensor_noise, X_train_tensor_clean)
test_set = torch.utils.data.TensorDataset(X_test_tensor_noise, X_test_tensor_clean)

In [11]:
BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

In [12]:
len(train_loader), len(test_loader)

(1875, 313)

In [13]:
def AE_model():
    return torch.nn.Sequential(
        # Encoder
        torch.nn.Conv2d(1, 32, (3, 3), padding='same'),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2, 2),
        torch.nn.Conv2d(32, 32, (3, 3), padding='same'),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2, 2),
        # Decoder
        torch.nn.ConvTranspose2d(32, 32, (2, 2), stride=2),
        torch.nn.ReLU(),
        torch.nn.ConvTranspose2d(32, 32, (2, 2), stride=2),
        torch.nn.ReLU(),
        torch.nn.Conv2d(32, 1, (3, 3), padding='same'),
        torch.nn.Sigmoid(),
    )

In [14]:
model = AE_model().cuda()
model

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): ConvTranspose2d(32, 32, kernel_size=(2, 2), stride=(2, 2))
  (7): ReLU()
  (8): ConvTranspose2d(32, 32, kernel_size=(2, 2), stride=(2, 2))
  (9): ReLU()
  (10): Conv2d(32, 1, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (11): Sigmoid()
)

In [15]:
optim = torch.optim.Adam(model.parameters())
loss_f = torch.nn.MSELoss()

In [16]:
wandb.init(project="mnist-denoise", entity="dean1t")

wandb: Currently logged in as: dean1t (use `wandb login --relogin` to force relogin)


In [16]:
def train(model, dataloader):
    model.train()
    total_loss = 0
    for cur_X, cur_Y in tqdm(dataloader, desc='Train'):
        optim.zero_grad()
        output = model(cur_X)
        loss = loss_f(cur_Y, output)
        loss.backward()
        optim.step()
        l = loss.detach().cpu()
        wandb.log({"train_loss": l})
        total_loss += l
    print('Train loss', total_loss / len(dataloader), flush=True)
    
def eval_model(model, dataloader):
    model.eval()
    total_loss = 0
    for cur_X, cur_Y in tqdm(dataloader, desc='Eval'):
        output = model(cur_X)
        loss = loss_f(cur_Y, output)
        l = loss.detach().cpu()
        wandb.log({"eval_loss": l})
        total_loss += l
    print('eval loss', total_loss / len(dataloader), flush=True)

In [18]:
n_epoch = 15
for i in range(n_epoch):
    print("Epoch", i, flush=True)
    train(model, train_loader)
    eval_model(model, test_loader)

Epoch 0


Train: 100%|██████████████████████████████████████████████████████████████████████| 1875/1875 [00:17<00:00, 109.86it/s]


Train loss tensor(0.1010)


Eval: 100%|█████████████████████████████████████████████████████████████████████████| 313/313 [00:00<00:00, 326.36it/s]

eval loss tensor(0.0157)
Epoch 1


...

eval loss tensor(0.0083)
Epoch 14



Train: 100%|██████████████████████████████████████████████████████████████████████| 1875/1875 [00:16<00:00, 114.98it/s]

Train loss tensor(0.0083)



Eval: 100%|█████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 239.43it/s]

eval loss tensor(0.0082)


In [106]:
i = np.random.randint(len(X_test_clean))
inp = X_test_tensor_noise[i]
gt = X_test_tensor_clean[i]
res = model(inp.unsqueeze(0))
draw_float(inp.detach().cpu().squeeze())
draw_float(gt.detach().cpu().squeeze())
draw_float(res.detach().cpu().squeeze())


In [50]:
wandb.finish()

eval_loss,█▇▇▇▅▅▅▅▄▃▅▆▆▅▃▅▁▄▃▅▂▃▃▂▃▅▅▂▂▄▃▄▃▂▁▂▃▅▄▃
train_loss,█▆▅▄▅▄▄▃▂▃▃▄▄▃▃▄▃▃▃▄▃▃▃▃▃▂▃▃▂▃▃▃▃▁▄▂▂▃▃▂
eval_loss,0.00766
train_loss,0.00862


In [107]:
torch.save(model.state_dict(), 'weights.pth')

In [108]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
0.weight 	 torch.Size([32, 1, 3, 3])
0.bias 	 torch.Size([32])
3.weight 	 torch.Size([32, 32, 3, 3])
3.bias 	 torch.Size([32])
6.weight 	 torch.Size([32, 32, 2, 2])
6.bias 	 torch.Size([32])
8.weight 	 torch.Size([32, 32, 2, 2])
8.bias 	 torch.Size([32])
10.weight 	 torch.Size([1, 32, 3, 3])
10.bias 	 torch.Size([1])


In [355]:
import struct

for param_tensor in model.state_dict():
    arr = model.state_dict()[param_tensor].detach().cpu().numpy()
    if param_tensor == '6.weight' or param_tensor == '8.weight':
        # need to transpose because transposed conv behaves differently
        arr = arr.transpose(1,0,2,3)
    l = list(arr.ravel())
    with open(f'model/{param_tensor}.bin', 'wb') as f:
        # just save binary files
        pack = struct.pack(f'{len(l)}f', *[float(cur) for cur in l])
        f.write(pack)


here
here
